In [1]:
import pandas
import numpy as np
import tensorflow as tf

tf.__version__

'1.4.1'

In [2]:
data = pandas.read_parquet('../data/to_send.pq')
print(data.shape)
data = data[['description', 'products']][pandas.notnull(data['products'])].copy().reset_index(drop=True)
print(data.shape)

(83897, 18)
(47516, 2)


In [3]:
data.head()

,description,products
0,Welch allyn combines its practical understandi...,"[power supply, body sub assy, medical, valve b..."
1,In line with the company s intention to ...,"[imo, advertising materials, point, imo label,..."
2,Services redaelli ricambi offers the ability t...,"[auto spare parts, tie rod, tie rod end, auto ..."
3,STROTHMANN not only delivers suitable mechanic...,"[covers non automated, demurrage rules form, r..."
4,"Established\nin 1991, tien jiang enterprise co...","[rubber, polyester, nylon, boot, support]"


# TEXT PROCESSING

In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from collections import Counter

# nltk.download('stopwords')

tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

data['products'] = data['products'].apply(lambda x: np.array([' '.join(tokenizer.tokenize(product)).lower()
                                                              for product in x.tolist()]))

all_products = []
for prod_list in data['products'][pandas.notnull(data['products'])].values:
    all_products += [' '.join(tokenizer.tokenize(product))
                     for product in prod_list.tolist()]
    
counter = Counter(all_products)
print('unique categories', len(counter.most_common()))

most_common = [product[0] for product in counter.most_common(600)]

def filter_categories(x):
    new_categories = np.array([product 
                               for product in x.tolist()
                               if product in most_common])
    if new_categories.shape[0] == 0:
        return np.nan
    return new_categories

lemma = nltk.wordnet.WordNetLemmatizer()
stopwords_cached = stopwords.words('english')

def filter_descriptions(text):
    cleaned_text = [token#lemma.lemmatize(token)
                      for token in tokenizer.tokenize(text.lower())
                      if token not in stopwords_cached]
    if len(cleaned_text) == 0:
        return np.nan
    return ' '.join(cleaned_text)

data['products'] = data['products'].apply(filter_categories)
data['description'] = data['description'].apply(filter_descriptions)

cleaned_data = data[(data['products'].notnull()) & (data['description'].notnull())].copy().reset_index(drop=True)
cleaned_data.shape

unique categories 71111


(35461, 2)

In [ ]:
cleaned_data['products'].values

In [5]:
cleaned_data.head()

,description,products
0,welch allyn combines practical understanding c...,"[power supply, medical]"
1,line company intention support international g...,"[imo, point, marine pollutant]"
2,services redaelli ricambi offers ability produ...,"[auto spare parts, auto spare, spare parts]"
3,strothmann delivers suitable mechanical system...,[line]
4,established tien jiang enterprise co ltd one s...,"[rubber, polyester, nylon, support]"


### fasttext model

In [6]:
from gensim.models.wrappers.fasttext import FastText

model = FastText.load_fasttext_format('../models/wiki.simple.bin')

In [7]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import MultiLabelBinarizer


nltk.download('stopwords')

# tokenizer = RegexpTokenizer(r'\w+')
tokenizer = RegexpTokenizer(r'[a-z]+')
lemma = nltk.wordnet.WordNetLemmatizer()
stopwords_cached = stopwords.words('english')

def get_embedding_matrix(text, first_n=50):
#     print(text)
    matrix = []
    text = [token#lemma.lemmatize(token)
            for token in tokenizer.tokenize(text)]
    for word in text[:first_n]:
        try:
            word_embedding = model.wv[word]
            matrix.append(model.wv[word])
        except KeyError:
            print("error in word", word)
            matrix.append(np.zeros(300))
    
    matrix = np.array(matrix)
    # fill text embeddings with seq_len < first_n with zeros
#     print(matrix.shape[0])
    if matrix.shape[0] < first_n:
        matrix = np.vstack((matrix, np.zeros((first_n - matrix.shape[0], 300))))
    return matrix

embeddings = []
for text in cleaned_data['description'].values:
    embeddings.append(get_embedding_matrix(text))
#     print(embeddings[-1].shape)
    
# onehot_y_df = pandas.get_dummies(data['severity'])

train_data = np.array(embeddings)

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([item.tolist() for item in cleaned_data['products'].values])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dimaStolpakov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
error in word zc
error in word zc
error in word vz
error in word vz
error in word vz


# LSTM

In [8]:
class DataBatcher():
    def __init__(self, X_, y_, batch_size_=30):
        self.X_ = X_
        self.y_ = y_
        self.batch_size_ = batch_size_
        self.from_ = 0
        self.to = self.from_ + self.batch_size_

    def next_batch(self):
        if self.to == len(self.X_):
            f = self.from_
            t = self.to
            self.from_ = 0
            self.to = self.to = self.from_ + self.batch_size_
            return self.X_[f:t], self.y_[f:t]
        elif self.to > len(self.X_):
            f = self.from_
            t = self.to - len(self.X_)
            self.from_ = self.to - len(self.X_)
            self.to = self.from_ + self.batch_size_
            return np.append(self.X_[f:], self.X_[:t], axis=0), np.append(self.y_[f:], self.y_[:t], axis=0)
        else:
            f = self.from_
            t = self.to
            self.from_ = self.to
            self.to = self.from_ + self.batch_size_
            return self.X_[f:t], self.y_[f:t]

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
from sklearn.model_selection import train_test_split

graph = tf.Graph()

with graph.as_default():

    # Training Parameters
    learning_rate = 0.001
    training_steps = 300000
    batch_size = 50
    display_step = 500
    
    # Network Parameters
    num_input = 300 # MNIST data input (img shape: 28*28)
    timesteps = 50 # timesteps
    num_hidden = 128 # hidden layer num of features
    num_classes = 600 # MNIST total classes (0-9 digits)
    num_layers = 2
    input_keep_prob = 0.6
    output_keep_prob = 1
    cell = tf.nn.rnn_cell.BasicLSTMCell

    # tf Graph input
    X = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])

    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([2 * num_hidden, num_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([num_classes]))
    }

    def RNN(x, weights, biases):

        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, timesteps, n_input)
        # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

        # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
        x = tf.unstack(x, timesteps, 1)

        # Cerate forward and backward cells
        cells_fw = [cell(num_hidden) for _ in range(num_layers)]
        cells_bw = [cell(num_hidden) for _ in range(num_layers)]
        
        # Add dropout
        cells_fw = [tf.contrib.rnn.DropoutWrapper(cell, 
                                                  input_keep_prob=input_keep_prob,
                                                  output_keep_prob=output_keep_prob) for cell in cells_fw]
        cells_bw = [tf.contrib.rnn.DropoutWrapper(cell, 
                                                  input_keep_prob=input_keep_prob,
                                                  output_keep_prob=output_keep_prob) for cell in cells_bw]
        
        outputs, _, _ = tf.contrib.rnn.stack_bidirectional_rnn(
            cells_fw=cells_fw,
            cells_bw=cells_bw,
            inputs=x,
            dtype=tf.float32)
        
#         rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
        
        return tf.matmul(outputs[-1], weights['out']) + biases['out']
        

    logits = RNN(X, weights, biases)
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

# Start training
with tf.Session(graph=graph) as sess:
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    # Run the initializer
    sess.run(init)
    batcher = DataBatcher(X_train, y_train, batch_size_=batch_size)

    for step in range(1, training_steps+1):
        batch_x, batch_y = batcher.next_batch()
        # Reshape data to get 28 seq of 28 elements
#         batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Loss= " + \
                  "{:.4f}".format(loss) + ", Train Acc= " + \
                  "{:.3f}".format(acc) +  ", Test Acc = " + \
                  "{:.3f}".format(sess.run(accuracy, feed_dict={X: X_test, Y: y_test})))

    print("Optimization Finished!")

#     # Calculate accuracy for 128 mnist test images
#     print("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Loss= 14.3337, Train Acc= 0.000, Test Acc = 0.001
Step 500, Loss= 13.2921, Train Acc= 0.020, Test Acc = 0.012
Step 1000, Loss= 13.0609, Train Acc= 0.000, Test Acc = 0.012
Step 1500, Loss= 11.9343, Train Acc= 0.000, Test Acc = 0.013
Step 2000, Loss= 12.4185, Train Acc= 0.000, Test Acc = 0.012


In [ ]:
'steell' in most_common

In [ ]:
' '.join(['qwe', '', 'rew'])

In [ ]:
unique_sets = []

for prod_list in data['products']:
    if set(prod_list) not in unique_sets:
        unique_sets.append(set(prod_list))
        
len(unique_sets)